### KoChatGPT 만들기

Foundational model 변경: gpu 메모리 및 이슈로 gpt-trinity 1.2B 사용해보기

이슈: torch 버전도 안맞고 뭐도 안맞고 아니 뭐 하나 할때마다 버전 충돌이 자꾸 일어남

심지어 아이펠에서 했는데, 모델 좀 바꿔보고 하려고 했더니만 당장은 뭐가 안되네?

일단 보류

플로우만이라도 최종본에 따보자

회고:

이게 이론상 RLHF를 적용해서 뭔가 파인튜닝도 하고, 특히 사람의 피드백이 들어가는 걸 체험할 수 있으니 좋을거라고 생각하는데

실제로는 강화학습 도입에 대해서 중간에 큰 갭이 있어서 코드 구현이든 뭐든 따라가기 갑자기 벅찬 느낌이 있음

#### 라이브러리 설치

In [2]:
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.28.1
!pip install colossalai==0.2.7
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .
%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1


import torch


Found existing installation: torch 1.9.1+cu111
Uninstalling torch-1.9.1+cu111:
  Successfully uninstalled torch-1.9.1+cu111
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp39-cp39-linux_x86_64.whl (1977.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1 requires pillow!=8.3.*,>=5.3.0, but you have pillow 8.3.2 which is incompatible.
fatal: destination path 'KoChatGPT' already exists and is not an empty directory.
/aiffel/KoChatGPT/colossalai_ChatGPT_230319
Processing /aiffel/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done
  Created wheel for chatgpt: filename=chatgpt-0.1.0-py3-none-any.whl size=46664 sha256=8d666fd7056587c924d26816e46adc50e06b66635074d657ebb25df2987e6ae0
  Stored in directory: /tmp/p

In [3]:
# import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

In [4]:
# parser 만들기
ps = argparse.ArgumentParser()
ps.add_argument('--data_path_1_SFT', type=str, default='./data_kochatgpt/kochatgpt_1_SFT.jsonl')
ps.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
ps.add_argument('--max_epochs', type=int, default=2)
ps.add_argument('--train_batch_size', type=int, default=8)
ps.add_argument('--output_dir', type=str, default='./output_1_SFT')

args = ps.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'
args.max_epochs = 1

print(args)

Namespace(data_path_1_SFT='./data_kochatgpt/kochatgpt_1_SFT.jsonl', model_name='skt/kogpt2-base-v2', max_epochs=1, train_batch_size=8, output_dir='./output_1_SFT')


In [5]:
## test & load skt gpt2 kroean
import torch
from transformers import GPT2LMHeadModel 
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '나는 자랑스런 세계인'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id,
                         bos_token_id=tokenizer.bos_token_id,
                         use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)
generator(
    ["0 : **는 게임 좋아하니\n1 :",
    "0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 :",
    "0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : "],
    **generation_args
)

/opt/conda/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

나는 자랑스런 세계인”이라며 “앞으로도 한국이 더 많은 것을 배우고 배울 수 있도록 노력하겠다”고 말했다.</d> 서울중앙지검 특수1부(부장검사 임관혁)는 이날 오전 10시30분부터 오후 1시까지 김 전 차관을 피의자 신분으로 불러 조사하고 있다.
김씨는 지난해 11월 검찰에 출석하면서 "검찰에서 (박근혜) 대통령과의 관계를 잘 모른다고 진술했다"고 말한 바 있다. 만약 박 대통령이 김씨에게 '최순실 게이트' 관련 내용을 얘기했다면 이는 거짓말이다.
하지만 이 같은 사실이 알려지면서 청와대 측은 사실무근의 입장을 밝혔다.
청원경찰


/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[[{'generated_text': '0 : **는 게임 좋아하니\n1 : ***는 게임을 싫어하니\n'}],
 [{'generated_text': '0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 : 그거 뭐야?!!!!!\n2 : 아, 그런 거 없잖아.\n3 : 다짜고짜...\n3 : 아니야, 이게 진짜 너희들 장난이야?!!!\n4 : 이건 또 누가 봐도 정신나간짓이야!!!\n5 : 안돼~!'}],
 [{'generated_text': '0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : 뭘 잘못했어?\n2 : 뭐가 잘못됐어?\n3 : 뭔 일이 있었어?\n4 : 뭐가 잘못된 거야?\n5 : 무슨 일이 있었던 거야?\n6 : 뭐라고 했어?\n7 : 아무 일도 없었어?\n8 : 뭐라고 한 거야?\n9 : 뭐라고 했는데'}]]

In [6]:
print(tokenizer.pad_token_id, tokenizer.pad_token)
print(tokenizer.eos_token_id, tokenizer.eos_token)
print(tokenizer.bos_token_id, tokenizer.bos_token)
print(tokenizer.unk_token_id, tokenizer.unk_token)

3 <pad>
1 </s>
1 </s>
5 <unk>


In [7]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "<pad>"
DEFAULT_EOS_TOKEN = "</s>" # 
DEFAULT_BOS_TOKEN = "</s>" # 위랑 같음
DEFAULT_UNK_TOKEN = "<unk>" 

# setting below
PROMPT_DICT = {
    "prompt_input": ("명령어:\n{prompt}\n\n### 입력:\n{input}\n\n### 응답:"
    ),
    "prompt_no_input": ("명령어:\n{prompt}\n\n### 응답:"
    ),
}

In [8]:
## 모델 준비
model = AutoModelForCausalLM.from_pretrained(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right", # 오른쪽을 채우기
    model_max_length=256,    
)
tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
        "pad_token": DEFAULT_PAD_TOKEN,
    }
)    
#tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=256, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)


In [11]:
## prepare data
from typing import Optional, Dict, Sequence
    
class SFT_dataset(Dataset):
    '''SFT dataset by wygo'''
    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=True):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")
        
        ## format
        pattern_instruction = 'prompt'
        pattern_input = 'input'
        pattern_output = 'completion' 

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)
            if verbose:
                print('## data check ##')
                print((list_data_dict[0]))
                
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]  # 템플릿 가져오기

        # 입력
        sources = []
        for example in list_data_dict:
            if example.get(pattern_input, "") != "":
                tmp = prompt_input.format_map(example)
            else:
                tmp = prompt_no_input.format_map(example)
            sources.append(tmp)
            
        # 출력
        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")

        if verbose:
            idx = 0
            print((sources[idx]))
            print((targets[idx]))
            print("Tokenizing inputs... This may take some time...")

    
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source만
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = IGNORE_INDEX  # source 부분은 -100으로 채운다

        data_dict = dict(input_ids=input_ids, labels=labels)        
        
        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))    
        
    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        """Tokenize a list of strings."""
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )        
        
        
    def __len__(self):
        return len(self.input_ids)

    
    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


train_dataset = SFT_dataset(data_path_1_SFT=args.data_path_1_SFT, tokenizer=tokenizer)
eval_dataset  = None 
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

## data check ##
{'prompt': '불고기용 고기 한우에요?', 'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.", 'tokens': 193}
명령어:
불고기용 고기 한우에요?

### 응답:
'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>
Tokenizing inputs... This may take some time...


In [12]:
!pip install --upgrade accelerate

     |████████████████████████████████| 345 kB 8.0 MB/s            
     |████████████████████████████████| 766.7 MB 1.9 kB/s              
     |████████████████████████████████| 207.5 MB 7.9 kB/s              
     |████████████████████████████████| 127.9 MB 52 kB/s              
     |████████████████████████████████| 664.8 MB 2.7 kB/s              
     |████████████████████████████████| 150.1 MB 2.5 kB/s             
     |████████████████████████████████| 21.1 MB 29 kB/s              
     |████████████████████████████████| 363.4 MB 1.6 kB/s              
     |████████████████████████████████| 211.5 MB 12 kB/s              
     |████████████████████████████████| 6.2 MB 89.8 MB/s            
     |████████████████████████████████| 13.8 MB 59.2 MB/s            
     |████████████████████████████████| 99 kB 12.9 MB/s            
     |████████████████████████████████| 253.1 MB 21 kB/s              
     |████████████████████████████████| 24.6 MB 34.0 MB/s            
     |███████

In [13]:
training_args = TrainingArguments(
    output_dir="./test",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps = 3,
    save_steps=500,
    warmup_steps=5,
    prediction_loss_only=True,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
trainer.save_state()

def save_model(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)

save_model(trainer=trainer, output_dir=args.output_dir)

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.144200
1000,2.939900
1500,2.875700
2000,2.756800
2500,2.738400
3000,2.659100


In [27]:
cd aiffel

/aiffel


In [28]:
ls

'=1.4.1'           Dockerfile   output_1_SFT/           setup.py
 aiffel/           examples/    pytest.ini              test/
 benchmarks/       img/         README.md               tests/
 chatgpt/          KoChatGPT/   requirements-test.txt   Untitled.ipynb
 data_kochatgpt/   LICENSE      requirements.txt        version.txt


In [30]:
## 모델 불러오기
if True:
  model = GPT2LMHeadModel.from_pretrained('./output_1_SFT')
model.eval()

## 추론 테스트
with torch.no_grad():
  generator = pipeline('text-generation', model=args.output_dir, tokenizer=tokenizer)
  # generator = pipeline('text-generation', model=model.cpu(), tokenizer=tokenizer, config={'max_length':800})

  generation_args = dict(
      num_beams=4,
      repetition_penalty=2.0,
      no_repeat_ngram_size=4,
      eos_token_id=375, # \n
      max_new_tokens=64,
      do_sample=True,
      top_k=50,
      early_stopping=True
  )

  list_prompt = ['불고기용 고기 한우에요?',
                '리처드 닉슨이 43대 부통령직을 수행한 연도는?',
                '시카고 오헤어 국제공항은 어디에 있어',
                '오늘 미세먼지 어때?']
  list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

  list_result = generator(list_prompt, **generation_args)
  for prompt, result in zip(list_prompt, list_result):
      print(('#'*70))
      print(('completion: %s'%(result[0]['generated_text'])))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


######################################################################
completion: 명령어:
불고기용 고기 한우에요?

### 응답:'저는 AI 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 일반적으로 불고기는 건강에 좋은 식품 중 하나이기 때문에 쇠고기를 먹는 것은 위험하지 않습니다. 그러나 일부 음식점에서는 불고기를 활용한 다양한 메뉴를 제공하고 있으니 참고하시기 바랍니다. important provide
######################################################################
completion: 명령어:
리처드 닉슨이 43대 부통령직을 수행한 연도는?

### 응답:'리처드 닉슨은 40대 부통령직을 수행했습니다. Now what you please provide more context. "리처드 닉슨"이 어떤 인물인지 명확하지 않아 정확한 답변을 드리기 어렵습니다. 추가적인 정보를 제공해주시면 더 정확한 답
######################################################################
completion: 명령어:
시카고 오헤어 국제공항은 어디에 있어

### 응답:'시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다. Canada OpenAir 국제공항, New York California 국제공항 등 다양한 국제공항이 운영되고 있습니다. National Geographic Center of Translation Systems, Director for the Commission of Korea
######################################################################
completion: 명령어:
오늘 미세먼지 어때?

### 응답:'저는 인공지능 챗봇이므로 미세먼지 정보를 알 수 없습니다. 하지만 일반적

___
학습완료! 256 토큰 내에서 어느정도 사람의 질문에 대한 답변을 꽤 잘 해내는 것 같은데?

In [ ]:
# 다음 모델 학습을 위해 할당한 GPU 메모리 초기화
torch.cuda.empty_cache()

### RM 모델 학습

In [37]:
!pip install "$HOME/aiffel/KoChatGPT/colossalai_ChatGPT_230319/"

Processing ./aiffel/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done
  Created wheel for chatgpt: filename=chatgpt-0.1.0-py3-none-any.whl size=46664 sha256=92afc11461857d6892e18783d3d882641166b5e32f6d919f088b0470517b38de
  Stored in directory: /tmp/pip-ephem-wheel-cache-0xlws0wt/wheels/79/25/c3/338e0c56a2253a8ea6c41e8692f6eb2409a3898c63b234b103
Successfully built chatgpt
  Attempting uninstall: chatgpt
    Found existing installation: chatgpt 0.1.0
    Uninstalling chatgpt-0.1.0:
      Successfully uninstalled chatgpt-0.1.0


In [39]:
torch.__version__

'1.13.1+cu116'

In [38]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 from torch.optim import Adam                                                                │
│    7 from chatgpt.dataset import RewardDataset                                                   │
│    8 from chatgpt.models.base import RewardModel                                                 │
│ ❱  9 from chatgpt.trainer import RewardModelTrainer                                              │
│   10 from chatgpt.trainer.strategies import NaiveStrategy                                        │
│   11 from datasets import load_dataset                                                           │
│   12 from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig         │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/__init__.py:1 in <module>                                                │
│                                                                                                  │
│ ❱ 1 from .base import Trainer                                                                    │
│   2 from .ppo import PPOTrainer                                                                  │
│   3 from .rm import RewardModelTrainer                                                           │
│   4                                                                                              │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/base.py:11 in <module>                                                   │
│                                                                                                  │
│     8 from torch.utils.data import DistributedSampler                                            │
│     9 from tqdm import tqdm                                                                      │
│    10                                                                                            │
│ ❱  11 from .callbacks import Callback                                                            │
│    12 from .strategies import Strategy                                                           │
│    13 from .utils import is_rank_0                                                               │
│    14                                                                                            │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/callbacks/__init__.py:3 in <module>                                      │
│                                                                                                  │
│   1 from .base import Callback                                                                   │
│   2 from .performance_evaluator import PerformanceEvaluator                                      │
│ ❱ 3 from .save_checkpoint import SaveCheckpoint                                                  │
│   4                                                                                              │
│   5 __all__ = ['Callback', 'PerformanceEvaluator', 'SaveCheckpoint']                             │
│   6                                                                                              │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/callbacks/save_checkpoint.py:4 in <module>                               │
│                                                                                                  │
│    1 import os                                             

In [32]:
class GPTRM_custom(RewardModel):
    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0, # 학습에 lora 적용시 rank 수
                 lora_train_bias: str = 'none',
                 tokenizer=None # 토크나이저
                ) -> None:
        
        if pretrained is not None: 
            # 지정된 pre-trained 모델 불러오기
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
            
        elif config is not None: 
            # 지정된 config 적용해 GPT2 모델 생성
            model = GPT2Model(config)
            
        else:
            # 기본 설정으로 GPT2 모델 생성
            model = GPT2Model(GPT2Config())
            
        if checkpoint:
            # checkpoint 불러오기
            model.gradient_checkpointing_enable()

        # reward가 스칼라값으로 나오도록 Head 설정
        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        """
        학습한 Reward Model 가중치 저장하기
        """
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [ ]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

# RM 불러오기
with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

### 데이터셋 만들기

In [ ]:
with open(HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    # 둘 중 랭킹이 높은 (값이 낮은) 답변을 chosen으로 저장
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # 이하 동문
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # 이하 동문
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)

    total_data_ranking2chosen.extend(one_data_ranking2chosen)

# RM 데이터셋 가공 전 크기
print('before data num: %d'%(len(list_data_dict)))
# RM 데이터셋 가공 후 크기
print('after  data num: %d'%(len(total_data_ranking2chosen)))
# RM 데이터셋 가공된 샘플
print('data example: \n%s'%total_data_ranking2chosen[45])

In [ ]:
# train/test split
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

# train/test 데이터셋 크기
print(len(train_data))
print(len(eval_data))

# RewardDataset으로 Wrapping
train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

In [ ]:
# RewardDataset 확인하기 
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

In [ ]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

trainer.fit(use_lora=0)

model.save_pretrained('aiffel/KoChatGPT/output_2_RM')

In [ ]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

In [ ]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

In [ ]:
# 다음 모델 학습을 위해 할당한 GPU 메모리 초기화
torch.cuda.empty_cache()

### PPO 강화학습 !!

In [ ]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [ ]:
with NaiveStrategy().model_init_context():
    # actor, 문장을 생성할 SFT된 모델
    actor = GPTActor(pretrained=HOME_DIR + '/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    # critic, 생성된 문장을 reward 값으로 평가할 RM
    critic = GPTCritic(pretrained=HOME_DIR + '/aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    # 비교군 모델
    initial_model = deepcopy(actor)
    # PPO로 학습할 모델
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [ ]:
# actor  옵티마이저
actor_optim = Adam(actor.parameters(), lr=5e-6)
# critic 옵티마이저
critic_optim = Adam(critic.parameters(), lr=5e-6)

(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

### PPO 데이터

In [ ]:
with open(HOME_DIR + '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [ ]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')

In [ ]:
# 답변 생성 함수
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

# 지시문
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

# 테스트용 질문 문장
list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

# 질문 문장에 지시문 추가
list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

# 질문에 따른 답변 생성
for input_text in list_prompt:
    output = generation(input_text)